In [10]:
from sys import stdout
 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import scale 

from keras.models import Sequential
from keras.layers import Dense

from progressbar import ProgressBar

In [11]:

#import training datadet

inter = pd.ExcelFile('training data_Github.xlsx')
inter = pd.read_excel(inter,'Combined')
#print(inter.info())
comb = inter.dropna(axis=0)
#print(comb.info())
comb = comb[comb["tR (min)"] != "?"]
#preprocess
for col in comb.columns[1:]:
    if (comb[col] == 0).all():
        comb = comb.drop([col], axis = 1)
comb = comb.reset_index(drop=True)
print(comb.info())

y = {'tR (min)': [round(x,4) for x in comb['tR (min)']]}
y = pd.DataFrame(comb, columns = ['tR (min)'])
print(y.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1790 entries, 0 to 1789
Columns: 145 entries, Project to fsulfone
dtypes: float64(101), int64(41), object(3)
memory usage: 2.0+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1790 entries, 0 to 1789
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tR (min)  1790 non-null   float64
dtypes: float64(1)
memory usage: 14.1 KB
None


In [14]:
#READ DATA
X = comb.drop(['Project','Column','Compound','tR (min)'], axis=1)
y = np.asarray(y).astype('float32')
X = np.asarray(X).astype('float32') 


test_pH=pd.unique(comb['PH'].tolist())
#print(test_column)
print(test_pH)
y_LOO_pred=[]
y_LOO_mea=[]
LABEL=[]
# for input
test_column='Zorbax Extend C18'  #Or change it to BEH C18 UPLC

temp= (comb['Column']==test_column) & (comb['PH']==4.8)
#print(temp)
X_test = X[temp,:]
X_train= X[~temp,:]
y_train= y[~temp]
y_test= y[temp]

from sklearn.preprocessing import StandardScaler
XScaler=StandardScaler()
YScaler=StandardScaler()

# Storing the fit object for later reference (Pete learned from ChatGPT)
X_scaled=XScaler.fit_transform(X_train)
y_scaled=YScaler.fit_transform(y_train)

# Generating the standardized values of X and y (Pete learned from ChatGPT)
X_test_scaled=XScaler.transform(X_test)


# create ANN model
model = Sequential()
# Defining the Input layer and FIRST hidden layer, both are same!
model.add(Dense(units=5, input_dim=141, kernel_initializer='normal', activation='relu'))
# Defining the Second layer of the model
# after the first layer we don't have to specify input_dim as keras configure it automatically
model.add(Dense(units=10, kernel_initializer='normal', activation='relu'))
# Defining the Third layer of the model
model.add(Dense(units=10, kernel_initializer='normal', activation='relu'))
# The output neuron is a single fully connected node 
# Since we will be predicting a single number
model.add(Dense(1, kernel_initializer='normal'))
# Compiling the model
model.compile(loss='mean_squared_error', optimizer='adam')    
# Fitting the ANN to the Training set
model.fit(X_scaled, y_scaled ,batch_size = 50, epochs = 50, verbose=0)


# Generating Predictions on testing data
Predictions=model.predict(X_test_scaled)
Predictions=list(Predictions)


# Scaling the predicted Price data back to original price scale
Predictions=YScaler.inverse_transform(Predictions)

result = {
'test column':test_column,
'test pH':test_pH,
'RMSE':math.sqrt(mean_squared_error(y_test, Predictions))}
print(result)


y_LOO_pred = pd.DataFrame(np.asarray(Predictions).astype('float32'),columns = ['pred tR by ANN (min)'])
y_LOO_mea = pd.DataFrame(np.asarray(y_test).astype('float32'),columns = ['measured tR (min)'])

results_ANN = pd.concat([y_LOO_mea,y_LOO_pred],axis=1)

print(results_ANN.info())
results_ANN.to_csv('results_ANN_LOO_pH.csv')

[ 4.8  1.9 10. ]
2/2 [==============================] - 0s 0s/step
{'test column': 'BEH C18 UPLC', 'test pH': array([ 4.8,  1.9, 10. ]), 'RMSE': 0.9671090181747964}
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   measured tR (min)     50 non-null     float32
 1   pred tR by ANN (min)  50 non-null     float32
dtypes: float32(2)
memory usage: 528.0 bytes
None
